In [120]:
import os
import glob
import re
import pandas as pd
from dodfminer.extract.polished.acts.aposentadoria import Retirements
import numpy as np
from itertools import zip_longest
from itertools import chain
from collections import Counter

In [79]:
# ls data/dodfs_flat/

In [80]:
def get_reverse_date(s):
    date = re.search( r'((\d+)[-](\d+)[-](\d+))', s)
    date = date.group()
    date = '-'.join(date.split('-')[::-1])
    return date

def get_num(s):
    return int(re.search('(\d+)', s).group())

In [81]:
df = pd.DataFrame()

## Quantos DODFs baixados?

In [82]:
dodfs_txt = glob.glob('data/dodfs_flat/*.txt')
df['file'] = [i.split('/')[-1] for i in dodfs_txt]
df['date'] = [get_reverse_date(s) for s in df.file]
# df['date'] = pd.to_datetime(df.date)
df['num'] = [get_num(s) for s in df.file]
df.drop('file', axis=1, inplace=True)
len(dodfs_txt)

595

## Quantos tiveram aposentadoria detectada?

In [83]:
%%time
dodfs_ext = [Retirements(i, 'regex') for i in dodfs_txt]
df['has_act'] = [not i.data_frame.empty for i in dodfs_ext] 
dodfs_ext = [i for i in dodfs_ext if not i.data_frame.empty]
print(
    "TOTAL DE ARQUIVOS COM APOSENTADORIAS:",
    df[~df.has_act].shape[0]
)

TOTAL DE ARQUIVOS COM APOSENTADORIAS: 368
CPU times: user 11.9 s, sys: 166 ms, total: 12.1 s
Wall time: 12.1 s


In [84]:
douro = pd.read_csv('data/aposentadoria-ouro/aposentaria-ouro-®.csv',
            sep='®', engine='python', index_col=False,)
douro.shape

(5516, 19)

In [85]:
douro.groupby(['DATA_DODF', 'NUM_DODF']).count().shape

(227, 17)

In [86]:
gp_ouro = douro.groupby(
    ['DATA_DODF', 'NUM_DODF']).count().iloc[:, 0:0].index
gp_miner = df[df.has_act].groupby(
    ['date', 'num']).count().iloc[:, 0:0].index

In [87]:
len(gp_miner), len(gp_ouro)

(226, 227)

In [88]:
sminer = set(gp_miner)
souro = set(gp_ouro)

In [89]:
MATTERS = ['DATA_DODF', 'NUM_DODF']
matters = ['date', 'num']
df_dtnum = df[df.has_act][matters].groupby(matters).count()
df_dtnum.reset_index(inplace=True)

douro_dtnum = douro[MATTERS].groupby(MATTERS).count()
douro_dtnum.reset_index(inplace=True)
douro_dtnum.rename({'DATA_DODF':'date', 'NUM_DODF':'num'},
                   inplace=True, axis=1)

In [90]:
souro_date = set(douro_dtnum.date.astype(str))
sdf_date = set(df_dtnum.date.astype(str))

In [118]:
duplis = list(
    zip_longest(douro_dtnum.values, df_dtnum.values,
                fillvalue=('empty', 'empty')
               )
)
eq = [i for i in duplis if np.array_equal(*i)]
neq = [i for i in duplis if not np.array_equal(*i)]
len(eq), len(neq)

(28, 199)

In [130]:
ctr= Counter([tuple(i) for i in chain(*neq)])
diff = {k:v for (k,v) in ctr.items() if v == 1}
diff

{('2018-04-19', 75): 1,
 ('2018-06-20', 116): 1,
 ('2018-07-11', 130): 1,
 ('2018-08-02', 146): 1,
 ('2018-08-17', 157): 1,
 ('2019-02-15', 33): 1,
 ('2019-02-26', 16): 1,
 ('2019-03-07', 44): 1,
 ('2019-03-15', 19): 1,
 ('2019-03-27', 58): 1,
 ('2019-04-03', 63): 1,
 ('2019-04-17', 30): 1,
 ('2019-07-03', 123): 1,
 ('2019-07-05', 125): 1,
 ('2019-07-23', 47): 1,
 ('2019-07-25', 139): 1,
 ('2019-08-30', 165): 1,
 ('2019-09-02', 166): 1,
 ('2019-09-10', 172): 1,
 ('empty', 'empty'): 1}

In [131]:
len(diff)

20

In [128]:
douro_dtnum.set_index(['date', 'num']).loc[('2018-04-19', 75)]

Series([], Name: (2018-04-19, 75), dtype: float64)

In [129]:
df_dtnum.set_index(['date', 'num']).loc[('2018-04-19', 75)]

KeyError: '2018-04-19'

In [107]:
np.array_equal( duplis[0] , duplis[1])

False

In [108]:
np.array_equal(duplis[0][0], duplis[0][1])

True

In [54]:
df_dtnum.shape, douro_dtnum.shape

((226, 2), (227, 2))